<a href="https://colab.research.google.com/github/hiydavid/homl-learning/blob/main/tryouts/tensorboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorBoard Tutorial

* [Link](https://www.youtube.com/watch?v=k7KfYXXrOj0) to video tutorial
* [Link](https://github.com/aladdinpersson/Machine-Learning-Collection/tree/master/ML/TensorFlow/Basics/tutorial17-tensorboard) to code on Github as seen on tutorial

---
# Load libaries and data

In [ ]:
# starter code
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import io
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds
from tensorflow import keras

In [ ]:
# get data
(ds_train, ds_test), ds_info = tfds.load(
    "cifar10",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
# functions to normalize and augment images
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

def augment(image, label):
    if tf.random.uniform((), minval=0, maxval=1) < 0.1:
        image = tf.tile(tf.image.rgb_to_grayscale(image), [1,1,3])
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [ ]:
# set autotune and batch size
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

In [ ]:
# setup for training dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.map(augment)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

In [ ]:
# setup for test dataset
ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(BATCH_SIZE)
ds_test = ds_train.prefetch(AUTOTUNE)

In [ ]:
# establish class names
class_names = [
    "Airplane", 
    "Autmobile", 
    "Bird", 
    "Cat", 
    "Deer",
    "Dog",
    "Frog", 
    "Horse", 
    "Ship", 
    "Truck",
]

---
# Build and compile model

In [ ]:
# function to build model
def get_model():
    model = keras.Sequential([
        keras.layers.Input((32, 32, 3)),
        keras.layers.Conv2D(8, 3, padding="same", activation="relu"),
        keras.layers.Conv2D(16, 3, padding="same", activation="relu"),
        keras.layers.MaxPool2D((2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(10),
    ])
    return model

In [ ]:
# build model & compile
model = get_model()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [ ]:
# create callback
tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir="tb_callback_dir",
    histogram_freq=1,
)

In [ ]:
# fit model with callback
model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test, # not best, but doesnt matter in this example
    callbacks=[tensorboard_callback],
    verbose=2,
)

In [ ]:
# open tensorboard
%load_ext tensorboard
%tensorboard --logdir tb_callback_dir